# Misc experiments

## Multi-stream

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import panson as ps
from panson import bundle
from panson import data_gens

In [ ]:
import sc3nb as scn

In [ ]:
# start scsynth
sc = scn.startup()
# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

In [ ]:
sc.exit()

In [ ]:
sc.server.blip()

In [ ]:
sc.server.latency = 0.1

In [ ]:
class Sines(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
        
    def init_parameters(self):
        self.amp = 0.3
    
    @bundle
    def init_server(self):
        scn.SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")

    @bundle
    def start(self):
        # lag time is decided based on the frame rate
        self.value_synth = scn.Synth("s2", {"amp": 0, "lg": 0.03})
        self.sincos_synth = scn.Synth("s2", {"amp": 0, "lg": 0.03})

    @bundle
    def _process(self, row):
        self.value_synth.set(
            "amp", self.amp,
            # map the intensity of the AU in one octave range
            "freq", scn.midicps(scn.linlin(row["value"], 0, 1, 69, 81))
        )
        self.sincos_synth.set(
            "amp", self.amp,
            "pan", row["cos"],
            # map the intensity of the AU in one octave range
            "freq", scn.midicps(scn.linlin(row["sin"], 0, 1, 69+4, 81+4))
        )
        

In [ ]:
son = Sines()
son

In [ ]:
from functools import partial

rtdp = ps.RTDataPlayerMulti(
    30,
    [partial(data_gens.dummy_sin_gen, fps=15, timestamps=False), partial(data_gens.dummy_sin_cos_gen, timestamps=False)], son)
rtdp

## Concatenate series

In [ ]:
import pandas as pd

In [ ]:
s = pd.Series(range(10), index=range(10, 20))
s

In [ ]:
s.to_csv('log.csv', mode='a')

In [ ]:
from panson import data_gens

In [ ]:
gen1 = data_gens.dummy_sin_cos_gen()
gen2 = data_gens.dummy_sin_gen()

In [ ]:
s1 = next(gen1)
s1

In [ ]:
s2 = next(gen2)
s2

In [ ]:
s2.to

In [ ]:
pd.concat([s1, s2], verify_integrity=True)

In [ ]:
s.to_frame().transpose()

## Video

In [ ]:
import panson as ps

In [ ]:
rtvp = ps.RTVideoPlayer(width=1280, height=1024, fps=30)

In [ ]:
rtvp = ps.RTVideoPlayer()

In [ ]:
rtvp.record()

In [ ]:
rtvp.stop()

In [ ]:
rtvp.quit()

In [ ]:
rtvp.set_filename('record.avi')

In [ ]:
rtvp.get_reply()

**VideoPlayer**

In [ ]:
vp = ps.VideoPlayer("/home/michele/Desktop/Thesis/project/record-000.avi", fps=30)

In [ ]:
vp.seek_time(5)

In [ ]:
vp.quit()

## Live plots

In [ ]:
import panson as ps
import pandas as pd
%matplotlib qt5

In [ ]:
live_features = ps.live_features.LiveFeatureDisplay(['AU_04', 'AU_05'])

In [ ]:
live_features.feed(pd.Series([2,1], ['AU_04', 'AU_05']))

In [ ]:
import sys, os, time, random
import numpy as np

import sc3nb as scn

In [ ]:
sc = scn.startup()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt5

## TimedQueueSC PMSon with timeseries data and matplotlib

The following example illustrates howto create a continuous sonification with concurrent plotting the time in a plot

* This presumes time-indexable data
* a 'maximum onset' variable is maintained to shutdown the continuously playing synths when done
* note that the highlight will only replot the marker, required time is thus independent of the amount of data plotted in the other plot.

In [ ]:
ts = np.arange(0, 20, 0.01)
data = np.vstack((ts, 
                  np.sin(2.5*ts) + 0.01*ts*np.random.randn(np.shape(ts)[0]), 
                  0.08*ts[::-1]*np.cos(3.5*ts)**2)).T

In [ ]:
# create figure
fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager()
# mngr.window.setGeometry(1200, 0, 500, 400)

# create axis, plots
ax.clear()
plmarked, = ax.plot([], [], "r-", lw=1)
pldata1, = ax.plot(data[:,0], data[:,1], "-", ms=2) # create plot 1
pldata2, = ax.plot(data[:,0], data[:,2], "-", ms=2) # create plot 2

In [ ]:
# create the queue
queue = scn.TimedQueueSC()

def mapcol(row, stats, col, val_from, val_to):  # helper for mapping
    return scn.linlin(row[col], stats[col, 0], stats[col, 1], val_from, val_to)

def update_plot(t): 
    global fig, ax, pldata1, pldata2, plmarked, selected
    plmarked.set_data([t,t], [-10000, 10000])
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata1)
    ax.draw_artist(pldata2)
    ax.draw_artist(plmarked)
    fig.canvas.update()

stats = np.vstack((np.min(data, 0), np.max(data, 0))).T
selected = np.zeros(np.shape(data)[0], bool)

# parameter mapping sonification with GUI
delay = 0.5
rate = 2

t0 = time.time()
queue.put_msg(t0, "/s_new", ["s2", 1200, 1, 0, "amp", 0])
queue.put_msg(t0, "/s_new", ["s2", 1201, 1, 0, "amp", 0])

max_onset = 0
latest_gui_onset = 0
gui_frame_rate = 60

ts = []
for i, r in enumerate(data[::2, :]):
    ts.append(time.time()-t0)
    if i==0: tmin = r[0]
    onset = (r[0]-tmin)/rate
    freq  = scn.midicps( mapcol(r, stats, 1, 60, 70))
    freqR = 0.5 * scn.midicps( mapcol(r, stats, 2, 70, 80))

    # sonification
    tt = t0 + delay + onset
    if tt > max_onset: max_onset = tt
    bundler = scn.Bundler(tt)
    bundler.add(0, "/n_set", [1200, "freq", freq, "num", 4, "amp", 0.2, "pan", -1, "lg", 0])
    bundler.add(0, "/n_set", [1201, "freq", freqR, "num", 1, "amp", 0.1, "pan", 1])
    queue.put_bundler(tt-0.2, bundler)
    if tt > latest_gui_onset + 1/gui_frame_rate:  # not more than needed gui updates
        latest_gui_onset = tt
        queue.put(tt, update_plot, (r[0],), spawn=False)
queue.put_msg(max_onset, "/n_free", [1200])
queue.put_msg(max_onset, "/n_free", [1201])
              
# queue.join()
print(time.time()-t0)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from pandas import Series

import time

from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'qt5')


class LiveFeatureDisplay:

    def __init__(self):
        ts = np.arange(0, 20, 0.01)
        data = np.vstack((ts,
                          np.sin(2.5 * ts) + 0.01 * ts * np.random.randn(np.shape(ts)[0]),
                          0.08 * ts[::-1] * np.cos(3.5 * ts) ** 2)).T
        
        # create figure
        self.fig, self.ax = plt.subplots(1)
        mngr = plt.get_current_fig_manager()
        # mngr.window.setGeometry(1200, 0, 500, 400)

        # fig.show()

        # create axis, plots
        ax.clear()
        plmarked, = ax.plot([], [], "r-", lw=1)
        pldata1, = ax.plot(data[:, 0], data[:, 1], "-", ms=2)  # create plot 1
        pldata2, = ax.plot(data[:, 0], data[:, 2], "-", ms=2)  # create plot 2

    def feed(self, features: Series):

        

    def update_plot(self, t):
        # pldata1, pldata2, plmarked, selected
        # plmarked.set_data([t, t], [-10000, 10000])
        
        self.ax.draw_artist(self.ax.patch)
        self.ax.draw_artist(pldata1)
        self.ax.draw_artist(pldata2)
        self.ax.draw_artist(plmarked)
        self.fig.canvas.update()

## Record

In [ ]:
scn.Recorder??

In [ ]:
# use the Recording class to capture the output
recorder = scn.Recorder(path="my_record.wav")

with sc.server.bundler(send_on_exit=False) as bundler:
    recorder.start(0.1)
    # /s_new synth name, node id, add action (0 to head), target (1 default group), synth arguments...
    scn.Synth("s1", {"freq": 200, "dur": 1})
    bundler.wait(0.3)
    scn.Synth("s1", {"freq": 300, "dur": 1})
    recorder.stop(1.5)
    
bundler.messages()

## sc3nb startup

In [ ]:
import sc3nb as scn

If we start the default server in another kernel, this will connect to it using **remote**. Hence it will be considered non-local (it was not booted by this client).

In [ ]:
# start scsynth
sc = scn.startup(start_sclang=False)
# sc = scn.startup()

# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

# multiclient setup by default
sc.server.dump_tree()

If now we restart the notebook and execute the code again, **the original server will be killed** and the server will be started here.

This is a very weird behaviour and it could confuse users that need to restart a notebook during development.

In [ ]:
sc.server.query_tree()

In [ ]:
sc.server.default_group

In [ ]:
sc.server

## Bundles

Automatic bundle nesting:
* it can be useful for composite sonification

In [ ]:
with sc.server.bundler(send_on_exit=False) as bundler_outer:
    with sc.server.bundler() as bundler:
        sc.server.msg("/s_new", ["s2", -1, 1, 1,], bundle=True)
        bundler.wait(0.3)
        sc.server.msg("/n_free", [-1], bundle=True)
    
bundler_outer

## Thread behaviour

The threads outputs results in the lastly executed cell.

The cell return when the main thread finishes.

In [ ]:
import threading
import time

def worker():
    print(threading.current_thread().getName(), 'Starting')
    time.sleep(10)
    print(threading.current_thread().getName(), 'Exiting')


def my_service():
    print(threading.current_thread().getName(), 'Starting')
    time.sleep(5)
    print(threading.current_thread().getName(), 'Exiting')


t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker)  # use default name

w.start()
w2.start()
t.start()

# # wait until threads terminate
# w.join()
# w2.join()
# t.join()

In [ ]:
print('test')

## Misc

Save frame information as csv as expected by the videoviewer

In [ ]:
timestamps = df.timestamp.to_numpy()
np.savetxt('phone.avi.csv', timestamps, delimiter=',')

### Video

#### ipywidgets

In [ ]:
from ipywidgets import Video
video = Video.from_file('NRT_videos/phone-processed-son.mp4')
video

In [ ]:
from ipywidgets import Image
image = Image()

#### PyQt

The following cell enable PyQt5 event loop integration. This is done by opening a QApplication for your notebook. This means that we have to avoid creating QApplication objects in our code (and consequenctly to start the event loop).
* https://stackoverflow.com/questions/30606462/closing-a-pyqt-widget-in-ipython-notebook-without-using-sys-exit

We can run the following magic as many times as we want, but we have to avoid creating QApplications in our code.

In [ ]:
%gui qt5

from PyQt5 import QtWidgets, QtCore
import pyqtgraph as pg

In [ ]:
QtCore.QCoreApplication.instance()

The following disables IPython GUI event loop integration.

In [ ]:
# %gui

In [ ]:
# # GUI initialization
# window = QtWidgets.QWidget()

# imggv = pg.GraphicsView()
# viewbox = pg.ViewBox()
# viewbox.setAspectLocked()
# viewbox.invertY(True)

# imggv.setCentralItem(viewbox)

# img = pg.ImageItem(np.zeros((100, 100, 3)))  # Todo: 3 -> channel variable
# viewbox.addItem(img)

# layout = QtWidgets.QGridLayout()
# layout.addWidget(imggv, 0, 0, 4, 4)
# window.setLayout(layout)

# # window always on top
# window.setWindowFlag(QtCore.Qt.WindowStaysOnTopHint)

# window.show()

In [ ]:
from videoviewer import NBVideoViewer

vv = NBVideoViewer()
vv.start()

In [ ]:
import skvideo
import skvideo.io

frames = skvideo.io.vread('files/processed/phone.avi')
frames.shape

In [ ]:
# frame = frames[0, ::5, ::5, [0,1,2]].T
# img.setImage(frame)

In [ ]:
import ipywidgets as widgets

In [ ]:
au4_node_id = 1234
# instantiate synths
sc.server.msg("/s_new", ["s2", au4_node_id, 0, 0, "amp", 0])

In [ ]:
sc.server.free_all()

In [ ]:
max_ = 332

# out = widgets.Output()

def on_change(change):
    # with out:
    #    print(change.new)
    
    idx = change.new
    
    row = df.iloc[idx]

    # only "max" should be enough (to clip the top part to 0.3)
    amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")   # TODO: exponential mapping
    # map the intensity of the AU in one octave range
    freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))

    with sc.server.bundler(0.1) as bundler:
        sc.server.msg("/n_set", [au4_node_id, "amp", amp, "freq", freq], bundle=True)

    frame = frames[idx, ::5, ::5, [0,1,2]]
    vv.update(frame)
    
def on_play_stop(change):
    
    # when the video is paused
    if change.now == False:
        # silence all the synths
        with sc.server.bundler() as bundler:
            sc.server.msg("/n_set", [au4_node_id, "amp", 0], bundle=True)
    
        
slider = widgets.IntSlider(max=max_)

# can't play data with unregolar sampling rate
play = widgets.Play(
    max=max_,
    step=1,
    interval=33,
    disabled=False
)

play.observe(on_change, names='value')
play.observe(on_play_stop, names='_playing')

widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])

In [ ]:
def sonification_callback(event):
    frame_idx = event.cls.value
    
#     sonify_row(df.iloc[frame_idx])
    img.setImage(frames[frame_idx, ::5, ::5, [0,1,2]].T)